# Implementation of Finite Elements

This tutorial shows how to implement our own finite elements in C++,
and how to use them within the NGSolve language. We implement first order and second order triangular finite elements.

* Finite elements implement the basis functions:
[myElement.hpp](/edit/myElement.hpp) 
[myElement.cpp](/edit/myElement.cpp)

* Differential operators define the mapping from coefficients to function values:
[myDiffOp.hpp](/edit/myDiffOp.hpp) 

* Finite element spaces implement the enumeration of degrees of freedom, and creation of elements:
[myFESpace.hpp](/edit/myFESpace.hpp) 
[myFESpace.cpp](/edit/myFESpace.cpp)

* Everything is included from the main file [mymodule.cpp](/edit/mymodule.cpp)

We read the main cpp-File to a string, and let NGSolve call the compiler, and load the new library as a Python module:

In [ ]:
from ngsolve.fem import CompilePythonModule
from pathlib import Path

txt = Path('mymodule.cpp').read_text() 
m = CompilePythonModule(txt, init_function_name='mymodule', add_header=False)

In [ ]:
from netgen.occ import unit_square
from ngsolve import *
from ngsolve.webgui import Draw

mesh = Mesh(unit_square.GenerateMesh(maxh=0.2, quad_dominated=False))

We can now create an instance of our own finite element space:

In [ ]:
fes = m.MyFESpace(mesh, secondorder=True, dirichlet="left|bottom|top")

and use it within NGSolve such as the builtin finite element spaces:

In [ ]:
print ("ndof = ", fes.ndof)

In [ ]:
gfu = GridFunction(fes)
gfu.Set(x*y)

Draw (gfu)
Draw (grad(gfu)[0], mesh);

and solve the standard problem:

In [ ]:
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
f = LinearForm(1*v*dx).Assemble()
gfu.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec
Draw (gfu);

Draw basis functions:

In [ ]:
gfu.vec[:] = 0
gfu.vec[30] = 1
Draw (gfu)

Documenetation provided in the DocInfo structure is available in Python - help:

In [ ]:
help (m.MyFESpace)

**Exercises:**

Extend MyFESpace by the following 1st and 2nd order elements:
 * 1D finite elements (ET_SEGM), as needed for boundary conditions
 * quadrilateral elements (ET_QUAD), use geom.GenerateMesh(quad_dominated=True)
 * tetrahedral elements (ET_TET), test it for 3D domains
 
Implement 3rd order elements